In [66]:
#df_arranjosra Bibliotecas
import pandas as pd
import math

In [67]:
#Leitura parametros Iniciais
df_arranjos = pd.read_csv('D38999_arranjos.csv', ';')
df_crimp = pd.read_csv('D38999_crimp.csv', ';')

In [68]:
#Leitura Interface
df_interface = pd.read_csv('D38999_interface.csv', ';')

#Isola Emissividades
emi = []
emi_t =[['Emissividade', '#28', '#26', '#24','#22','#20','#18','#16','#14','#12']]
for i in (df_interface['Emissividade'].unique()):
    emi = []
    emi.append(i)
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#28'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#26'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#24'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#22'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#20'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#18'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#16'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#14'].sum())
    emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#12'].sum())
    emi_t.append(emi)
print(emi_t)

# Cria dataframe com emissividades
df = pd.DataFrame(emi_t)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header
trata_df = df

[['Emissividade', '#28', '#26', '#24', '#22', '#20', '#18', '#16', '#14', '#12'], ['A', 0, 7, 308, 0, 5, 6, 2, 0, 2], ['S', 0, 4, 0, 0, 5, 0, 0, 0, 0]]


In [69]:
trata_df['22D'] = trata_df['#28'] + trata_df['#26'] + trata_df['#24'] + trata_df['#22']
trata_df['#28'] = 0
trata_df['#26'] = 0
trata_df['#24'] = 0
trata_df['#22'] = 0
trata_df['20'] = trata_df['#20']
trata_df['#20'] = 0
trata_df['16'] = trata_df['#18'] + trata_df['#16']
trata_df['12'] = trata_df['#14'] + trata_df['#12']
tratad_df = trata_df.drop(columns=['#28', '#26', '#24', '#22', '#20', '#18', '#16', '#14', '#12'])

In [48]:
tratad_df.head()

,Emissividade,22D,20,16,12
1,A,315,5,8,2
2,S,4,5,0,0


,Emissividade,22D,20,16,12
1,A,315,5,8,2
2,S,4,5,0,0


In [62]:
trata_df.head()

,Emissividade,22D,20,16,12
1,A,315,5,8,2
2,S,4,5,0,0


In [56]:
def preparaProcura(self, tratad_df):
    tes = 0
    df_encontrado = pd.DataFrame(columns = ['Código', '22D','20','16','12','12(Coax)','8(coax)','8(Twinax)'])
    lista_arranjo = tratad_df.values.tolist()
    lista_total =[]
    lista_total = [['Emissividade', 'Bitola', 'Arranjo']]

In [63]:
trata_df

,Emissividade,22D,20,16,12
1,A,315,5,8,2
2,S,4,5,0,0


In [70]:
tes = 0
df_encontrado = pd.DataFrame(columns = ['Código', '22D','20','16','12','12(Coax)','8(coax)','8(Twinax)'])
lista_arranjo = tratad_df.values.tolist()
lista_total =[]
lista_total = [['Emissividade', 'Bitola', 'Arranjo']]

for i in lista_arranjo:
    
    # Encontra Bitola 22
    #Caso seja insuficiente o número de conexões para está bitola, a quantidade necessária de conectores maiores
    if (lista_arranjo[tes][1]) > df_arranjos['22D'].max():
        trataArranjo = math.ceil((lista_arranjo[tes][1])/df_arranjos['22D'].max())
        for i in range(trataArranjo-1):
            df_encontrado = df_arranjos.loc[df_arranjos['22D'] == (df_arranjos['22D'].max())]
            merda = df_encontrado['Código'].values.tolist()
            lista_total.append([lista_arranjo[tes][0],'22D', merda[0]])
            lista_arranjo[tes][1] = lista_arranjo[tes][1] - df_arranjos['22D'].max()
    
    #Trata valores em caso ser menor que 1 conector 
    if (lista_arranjo[tes][1]) !=0: # Acho que tinha essa bitola? Achou errado, caro otário
        df_encontrado = df_arranjos.loc[df_arranjos['22D'] == (lista_arranjo[tes][1])]
        for i in range(df_arranjos['22D'].max()): #Procura até a ultima configuração possível para está bitola
            if (df_encontrado.empty): # Se não encontrou, itera para um valor acima
                lista_arranjo[tes][1] = lista_arranjo[tes][1] + 1;
                df_encontrado = df_arranjos.loc[df_arranjos['22D'] == (lista_arranjo[tes][1])]
                
                #Separa se pode estar junto a outras bitolas ou não
                trataArranjo = df_encontrado.values.tolist()
                if not df_encontrado.empty:
                    if (trataArranjo[0][4] >0) or (trataArranjo[0][5] >0) or (trataArranjo[0][6] >0) or (trataArranjo[0][7] >0):
                        lista_arranjo[tes][1] = lista_arranjo[tes][1] + 1;
                        df_encontrado = df_encontrado.iloc[0:0]
        
        # Adiciona a lista a bitola selecionada
        if not df_encontrado.empty:
            merda = df_encontrado['Código'].values.tolist()
            lista_arranjo[tes].append(merda[0])
            lista_total.append([lista_arranjo[tes][0],'22D', merda[0]])   
        
    # Encontra Bitola 20
    #Trata resto
    if (lista_arranjo[tes][2]) !=0:
        df_encontrado = df_arranjos.loc[df_arranjos['20'] == (lista_arranjo[tes][2])]
        for i in range(df_arranjos['22D'].max()):
            if (df_encontrado.empty):
                lista_arranjo[tes][2] = lista_arranjo[tes][2] + 1;
                df_encontrado = df_arranjos.loc[df_arranjos['20'] == (lista_arranjo[tes][2])]
                if not (df_encontrado.empty):
                    break;
        
        if not df_encontrado.empty:
            merda = df_encontrado['Código'].values.tolist()
            lista_arranjo[tes].append(merda[0])
            lista_total.append([lista_arranjo[tes][0],'20', merda[0]]) 
    
    # Encontra Bitola 16
    #Trata resto
    if (lista_arranjo[tes][3]) !=0:
        df_encontrado = df_arranjos.loc[df_arranjos['16'] == (lista_arranjo[tes][3])]
        for i in range(df_arranjos['22D'].max()):
            if (df_encontrado.empty):
                lista_arranjo[tes][1] = lista_arranjo[tes][3] + 1;
                df_encontrado = df_arranjos.loc[df_arranjos['16'] == (lista_arranjo[tes][3])]
                if not (df_encontrado.empty):
                    break;
        
        
        merda = df_encontrado['Código'].values.tolist()
        lista_arranjo[tes].append(merda[0])
        lista_total.append([lista_arranjo[tes][0],'16', merda[0]]) 
    
    # Encontra Bitola 12
    #Trata resto
    if (lista_arranjo[tes][4]) !=0:
        df_encontrado = df_arranjos.loc[df_arranjos['12'] == (lista_arranjo[tes][4])]
        for i in range(df_arranjos['22D'].max()):
            if (df_encontrado.empty):
                lista_arranjo[tes][1] = lista_arranjo[tes][4] + 1;
                df_encontrado = df_arranjos.loc[df_arranjos['12'] == (lista_arranjo[tes][4])]
                if not (df_encontrado.empty):
                    break;
        
        if not df_encontrado.empty:
            merda = df_encontrado['Código'].values.tolist()
            lista_arranjo[tes].append(merda[0])
            lista_total.append([lista_arranjo[tes][0],'12', merda[0]])    
    
    #
    tes = tes+1

pd.DataFrame(lista_total)

,0,1,2
0,Emissividade,Bitola,Arranjo
1,A,22D,J35
2,A,22D,J35
3,A,22D,F35
4,A,20,B5
5,A,16,E8
6,A,12,C13
7,S,22D,B54
8,S,20,B5


## 

,0,1,2
0,Emissividade,Bitola,Arranjo
1,A,22D,J35
2,A,22D,J35
3,A,22D,F35
4,S,22D,B54


In [1]:
# Consulta rápida para todos arranjos de uma bitola
df_encontrado = df_arranjos.loc[df_arranjos['22D'] >= 10]
df_encontrado

NameError: name 'df_arranjos' is not defined

## math.ceil(200/df_arranjos['22D'].max())